https://docs.opencv.org/2.4/modules/contrib/doc/facerec/facerec_tutorial.html

In [3]:
!pip install opencv-contrib-python

In [4]:
import os
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

In [39]:
# Function to augment a single image
def augment_image(image):
    augmented_images = []

    # Flip the image
    flipped = cv2.flip(image, 1)  # 1 for horizontal flip
    augmented_images.append(flipped)

    # Rotate the image by 15 degrees
    height, width = image.shape[:2]
    center = (width // 2, height // 2)
    M = cv2.getRotationMatrix2D(center, 15, 1.0)  # Rotate by 15 degrees
    rotated = cv2.warpAffine(image, M, (width, height))
    augmented_images.append(rotated)


    return augmented_images

In [6]:
# Initialize the recognizer
recognizer = cv2.face.LBPHFaceRecognizer_create()

In [7]:
# Image paths (ensure these are correct)
imagePaths = '/content/Singers'

In [8]:
# Load the face cascade
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [22]:
X = []
ids = []
count = 0
label_dict = {}  # Dictionary to store labels and corresponding ids


# Loop over the image paths
for imageName in os.listdir(imagePaths):

    # Construct the full image path
    imagePath = os.path.join(imagePaths, imageName)

    # Read the image
    image = cv2.imread(imagePath)

    # Check if the image was loaded successfully
    if image is None:
        print(f"Error: Unable to load image {imagePath}. Skipping...")
        continue



    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=3,
        minSize=(30, 30)
    )

    # Process the detected faces
    for (x, y, w, h) in faces:
        # Extract the face ROI and append it to the training set
        face_roi=gray[y:y+h, x:x+w]
        X.append(face_roi)
        label = os.path.split(imagePath)[-1].split('.')[0]
        ids.append(count)
        label_dict[label] = count



        # Data Augmentation
        augmented_faces = augment_image(face_roi)
        for aug_face in augmented_faces:
            X.append(aug_face)
            ids.append(count)
        break  # Break after the first face is detected in each image
    count += 1

In [23]:
# Train the recognizer
recognizer.train(X, np.array(ids))

In [24]:
# Save the trained model
recognizer.save('cvtraining.yml')

# Testing

In [25]:
recognizer.read('cvtraining.yml')

In [26]:
testpath = "Selena.jpeg"

In [27]:
testpath = "test.webp"

In [33]:
testpath = "test.jpg"

In [34]:
# Read the test image
image = cv2.imread(testpath)


In [35]:

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


In [36]:

# Convert to numpy array for face detection
img_numpy = np.array(gray, 'uint8')


In [37]:

# Detect faces in the image
faces = faceCascade.detectMultiScale(
    img_numpy,
    scaleFactor=1.1,
    minNeighbors=10,
    minSize=(30, 30)
)

In [38]:
# Loop through the detected faces and predict using the recognizer
for (x, y, w, h) in faces:
    # Predict the label (ID) and confidence of the detected face
    id, confidence = recognizer.predict(gray[y:y+h, x:x+w])
    name = [key for key, value in label_dict.items() if value == id]
    print(f"Predicted ID: {id}")
    print(f"Predicted name: {name}")
    print(f"Confidence: {confidence}")

    # Draw a rectangle around the detected face (optional, for visualization)
    cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)
    cv2.rectangle(image, (x, y + h - 35), (x + w, y + h), (255, 0, 0), cv2.FILLED)
    font = cv2.FONT_HERSHEY_DUPLEX
    cv2.putText(image, name[0], (x + 6, y + h - 6), font, 1.0, (0, 0, 0), 1)

Predicted ID: 9
Predicted name: ['cardib']
Confidence: 34.66114210034632


In [ ]:
# Display the image with the detected face and drawn rectangle
cv2_imshow(image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Output hidden; open in https://colab.research.google.com to view.